In [1]:
import requests

API_KEY = "xxxxx"
MPAN = "xxxxx"
SERIAL = "xxxxx"

In [2]:
url = f"https://api.octopus.energy/v1/electricity-meter-points/{MPAN}/meters/{SERIAL}/consumption/?period_from=2020-12-24T00:00:00Z"
data = requests.get(url, auth=(API_KEY, ""))
print(data)

<Response [200]>


In [3]:
print(data.json())


{'count': 47, 'next': None, 'previous': None, 'results': [{'consumption': 0.189, 'interval_start': '2020-12-24T23:00:00Z', 'interval_end': '2020-12-24T23:30:00Z'}, {'consumption': 0.172, 'interval_start': '2020-12-24T22:30:00Z', 'interval_end': '2020-12-24T23:00:00Z'}, {'consumption': 0.119, 'interval_start': '2020-12-24T22:00:00Z', 'interval_end': '2020-12-24T22:30:00Z'}, {'consumption': 0.142, 'interval_start': '2020-12-24T21:30:00Z', 'interval_end': '2020-12-24T22:00:00Z'}, {'consumption': 0.155, 'interval_start': '2020-12-24T21:00:00Z', 'interval_end': '2020-12-24T21:30:00Z'}, {'consumption': 0.17, 'interval_start': '2020-12-24T20:30:00Z', 'interval_end': '2020-12-24T21:00:00Z'}, {'consumption': 0.178, 'interval_start': '2020-12-24T20:00:00Z', 'interval_end': '2020-12-24T20:30:00Z'}, {'consumption': 0.179, 'interval_start': '2020-12-24T19:30:00Z', 'interval_end': '2020-12-24T20:00:00Z'}, {'consumption': 0.214, 'interval_start': '2020-12-24T19:00:00Z', 'interval_end': '2020-12-24T19

In [4]:
from apd.sensors.base import HistoricalSensor, JSONSensor
import typing as t
import datetime
import pint
from pint import _DEFAULT_REGISTRY as ureg


In [5]:
class PowerUsageSensor(HistoricalSensor[t.Any], JSONSensor[t.Any]):
    def historical(
        self, start: datetime.datetime, end: datetime.datetime
    ) -> t.Iterable[t.Tuple[datetime.datetime, t.Any]]:
        url = f"https://api.octopus.energy/v1/electricity-meter-points/{MPAN}/meters/{SERIAL}/consumption/"
        data = requests.get(url, params={"period_from": start.isoformat(), "period_to": end.isoformat(), "page_size": 25000}, auth=(API_KEY, ""))
        results = data.json()["results"]
        for item in results:
            yield datetime.datetime.fromisoformat(item['interval_end'].strip("Z")), ureg.Quantity(item['consumption'], ureg.kilowatthour)


In [6]:
tuple(PowerUsageSensor().historical(datetime.datetime(2020,12,24,12,0,0), datetime.datetime(2020,12,24,23,0,0)))

((datetime.datetime(2020, 12, 24, 23, 30), 0.189 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 12, 24, 23, 0), 0.172 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 12, 24, 22, 30), 0.119 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 12, 24, 22, 0), 0.142 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 12, 24, 21, 30), 0.155 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 12, 24, 21, 0), 0.17 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 12, 24, 20, 30), 0.178 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 12, 24, 20, 0), 0.179 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 12, 24, 19, 30), 0.214 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 12, 24, 19, 0), 0.223 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 12, 24, 18, 30), 0.334 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 12, 24, 18, 0), 1.022 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 12, 24, 17, 30), 0.251 <Unit('kilowatt_hour')>),
 (datetime.datetime(2020, 12, 24